# Homework 4


# Diffusion and Brownian Motion

Read through M. Kozdron's paper on diffusion and the heat equation http://stat.math.uregina.ca/~kozdron/Research/UgradTalks/BM_and_Heat/heat_and_BM.pdf. By the 20'th century, heat was partially accepted to be governed by the random motion of particles elastically bumping into each other, with temperature being directly proportional to the particles velocity. **Diffusion** is a model of particle movement where we assume that the motion of particles are governed by a random process, that is for any time step $\Delta t$, the particle can be modeled as taking a "step" of some length in a random direct. We assume that molecules in a still gas or liquid move through a medium by diffusion, like dye spreading outwards through water. 

<img width=300 src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/12/Diffusion.svg/2880px-Diffusion.svg.png">

The diffusion of particles in a liquid (or gas) can be simulated computationally using a random walk. In a random walk, we discretize time into intervals $t_0, t_1, \ldots, t_N$ and construct a path by picking a starting point $x(0) = 0$ and compute each step by

$$
x(t_{i+1}) = x(t_i) + r_i\,,
$$

where $r_i$ is sampled from a probability distribution. For example, if you imagine flipping a coin over and over and taking a step forward for each heads and backwards for each tails your motion would follow a *binary* random walk since a each step $r_i$ is drawn from a binary distribution. 

## Simulating random walks

We want to simulate a random walk and compare it the the heat equation. Numpy has many distributions implemented in 
the `numpy.random` library. Since we are interested in checking Brownian Motion, we will draw each step from a random normal distribution $\mathcal{N}(0,1)$ with mean $\mu = 0$ and variance $\sigma^2 = 1$.

Assume that we want to generate a random walk starting from $x(0) = 0$ with $N = 100$ steps of size $dT$, with each step drawn independently from $\mathcal{N}(0,1)$. To generate a time vector we use

* `numpy.arange(start, stop, step)` - Returns a vector from `start` to `stop` with step `step`. In case the numbers don't line up exactly, the half open interval [`start`, `stop`) will be returned.

To generate samples from the normal distribution we use

* `np.random.normal(loc=0, scale=1, size=1)` - Returns a vector of `size` samples from the normal distribution with mean `loc` and variance `scale`. 

Then, we use a loop to generate the random walk for $x$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

dT = .1
N = 100
x = np.zeros(N)
t = np.arange(0,dT*N,step=dT)

x[0] = 0

for i in range(N-1):
    x[i+1] = x[i] + np.random.normal(loc=0, scale=1,)
    
plt.plot(t,x)

Given a vector `x`, the `np.cumsum(x)` returns the vector where each element is the cumulative sum of `x`. This allows us to shorten the code to the code below:

In [ ]:
dT = .1
N = 100
t = np.arange(0,dT*N,step=dT)

x = np.cumsum(np.random.normal(size=N))

plt.plot(t,x)

## Monte Carlo Simulation of Random Walks

We are interested in the long term distribution of random walks. A Monte Carlo method samples a random process many times and uses the result to explore the eventual outcome. The code below is an example, it uses a loop to generate 100 sample random walks and plots them on the same axis

In [ ]:
dT = .1
N = 100
t = np.arange(0,dT*N,step=dT)

M = 100
endpoints = np.zeros(M)

for i in range(M):
    plt.plot(t,np.cumsum(np.random.normal(size=N)))

Lets get a better idea of the distribution of the endpoints. The code below uses a loop to generate 10000 sample random walks, saves the end point in a vector `endpoinst` and then uses `plt.hist` to plot a histogram of the location of the last step. In the histogram we use `density=True` to scale the histogram so that it shows the probabilities of each walking ending in the corresponding interval.

In [ ]:
dT = .1
N = 100
t = np.arange(0,dT*N,step=dT)

M = 10000
endpoints = np.zeros(M)

for i in range(M):
    endpoints[i] = np.cumsum(np.random.normal(size=N))[N-1]
    
plt.hist(endpoints,bins=100,density=True)
plt.ylabel("Probability")

**Exercise:** Using a subplot, produce histograms for $t=1$, $t=5$ and $t=10$ both separately and all plotted on the same axis.

## Simulating PDEs

Lets recall the PDE for the time dependent diffusion equations: For a normal process with variance parameter $\sigma^2 = 1$, the heat equation is 

$$
\frac{\partial U}{\partial t} = \frac{1}{2} \frac{\partial^2 U}{\partial x^2}
$$

Instead of using a solver, we need to try to solve this equation by hand. That means approximating the derivatives, and the solution. To perform the update step we will use Eulars method: For a fixed small time step $\Delta t$, 

$$
\frac{\partial U}{\partial t} \approx \frac{U(t+\Delta t) - U(t)}{\Delta t}
$$

so 

$$
U(t+\Delta t) = U(t) + \Delta t \left( \frac{1}{2}\frac{\partial^2 U}{\partial x^2}\right)\,.
$$

We now need to simulate $\frac{\partial^2 U}{\partial x^2}$. We split the $x$ axis up into $K$ points, with uniform step given by $\Delta x$. At each point $x_j$, we could estimate the spacial derivative in one of three ways:

\begin{align}
\frac{\partial U}{\partial x}(x_i)_L &= \frac{U(x_i) - U(x_{i-1})}{\Delta x}\hspace{3 em} & \text{from the left}
\\
\frac{\partial U}{\partial x}(x_i)_R &= \frac{U(x_{i+1}) - U(x_{i})}{\Delta x}\hspace{3 em} & \text{from the right}
\\
\frac{\partial U}{\partial x}(x_i)_A &= \frac{U(x_{i+1}) - U(x_{i-1})}{2\Delta x}\hspace{3 em} & \text{average of the two}
\end{align}

We can think of these graphically by 

Let us take the average. The second derivative is then computed by repeating the process. For the second derivative, we will take the derivative by computing the rate of change of the slope as computed by from the left and the right

\begin{align}
\frac{\partial^2 U}{\partial x^2}(x_i) = \frac{\frac{\partial U}{\partial x}(x_i)_R - \frac{\partial U}{\partial x}(x_i)_L}{\Delta x}= \frac{U(x_{i+1}) -2U(x_i)+ U(x_{i-1})}{(\Delta x)^2}
\end{align}

There is a choice we have to make: We cannot evaluate this computation at either $x_0$ or $x_K$ since there is not $x_{-1}$ or $x_{K+1}$. We have to make a choice about what do at the ends. There are a couple of choices we could make, for example

* Fix $\frac{dU}{dt} = 0$ at the end points by fixing $\frac{\partial^2 U}{\partial x^2}(x_0) = \frac{\partial^2 U}{\partial x^2}(x_K) = 0$.

* Fix $\frac{\partial^2 U}{\partial x^2}(x_0)$ and $\frac{\partial^2 U}{\partial x^2}(x_K)$ to be constant by setting $\frac{\partial^2 U}{\partial x^2}(x_0) = \frac{\partial^2 U}{\partial x^2}(x_1)$ and $\frac{\partial^2 U}{\partial x^2}(x_K) = \frac{\partial^2 U}{\partial x^2}(x_{K-1})$.

We will use the second condition. 

In [ ]:
Dt = .1     # Set the time step
N = 100     # Set the number of time steps
K = 100     # Set the number of spacial slices

t = np.arange(0,dT*N,step=dT)               # Construct the vector of time points
x, Dx = np.linspace(-10,10,K,retstep=True)  # Constrcut the vector of spacial points

U = np.zeros(K)       # Define a vector U = U(t=0,x)
U[49:51] = .5         # We'll evenly distribute the function between the middle points
plt.plot(x,U)         # Plot the mass at t = 0

We can construct the vector $\frac{\partial^2 U}{\partial x^2}$ for each $x_i$ in one line carefully using Python's indexing conventions: Let `U` be a vector giving the "mass" of a gas at point the points $x_0$ up to $x_{K-1}$. 

* `U[2:]` is the vector $[U(x_2), U(x_3), \ldots, U(x_{K-1})]$
* `U[:-2]` is the vector $[U(x_0), U(x_1), \ldots, U(x_{K-3})]$
* `U[1:-1]` is the vector $[U(x_1), U(x_2), \ldots, U(x_{K-2})]$

We can then compute the vector 

\begin{align}
\frac{\partial^2 U}{\partial x^2}(x_i) = \frac{\frac{\partial U}{\partial x}(x_i)_R - \frac{\partial U}{\partial x}(x_i)_L}{\Delta x}= \frac{U(x_{i+1}) -2U(x_i)+ U(x_{i-1})}{(\Delta x)^2}
\end{align}

in a single line by using

`ddU[1:-1] = (U[2:] -2*U[1:-1]+ U[:-2])/(Dx**2)`

We can impose the boundary condition by using `ddU[0] = ddU[1]` and `ddU[-1] = ddU[-2]`. 

Finally, implement Eulars Method to compute 

for 

$$
\frac{\partial U}{\partial t} = \alpha^2 \frac{\partial^2 U}{\partial x^2}
$$

when $\alpha = \frac{1}{16}$.

In [ ]:
Dt = .1     # Set the time step
N = 100     # Set the number of time steps
K = 100     # Set the number of spacial slices

t = np.arange(0,dT*N,step=dT)               # Construct the vector of time points
x, Dx = np.linspace(-10,10,K,retstep=True)  # Constrcut the vector of spacial points

U = np.zeros(K)       # Define a vector U = U(t=0,x)
U[49:51] = .5/Dx      # We evenly distribute the function between the middle points, dividing by the distance
plt.plot(x,U)         # Plot the mass at t = 0

ddU = np.zeros(K)

for i in range(N):
                   # Compute ddU
                   # Impose Boundary Conditions
                   # Impose Boundary Conditions
    
                   # Use Eulars Method To Compute U at t+1
    
plt.plot(x,U)

## Problem 1:

#### Part a
The diffusion equation with parameter $\alpha$ matches the Brownian motion with variance given by $2\alpha^2\Delta t = \sigma^2$, or $\alpha = \frac{\sigma}{\sqrt{2\Delta t}}$. Generate the histogram of a Monte Carlo simulation of 10000 random walks with $\Delta t = .1$ after $N=100$ steps. Pick the variance $\sigma$ to match $\alpha^2 = 1/16$.

Plot both the histogram and the result of your simulation of the diffusion equation on the same plot. They should be almost exactly equal. Turn in this plot. 

#### Part b
The Eular Method breaks down for $\alpha^2 > 1/2$ and even gets problematic for $\alpha^2 > 1/5$. Plot both the histogram and the simulation of the diffusion equation when $\alpha^2 = 1/4$. Turn in this plot.  

# Analytic Problems:

## Problem 2:

Read Section 9.6 on radial heat conduction. Answer Question 12.3 in the book using Question 9.8. You must show your derivation. 

## Problem 3:

Answer Question 12.5 in the book. 

## Problem 4:

For $\frac{\partial \rho}{\partial t} + c\frac{\partial \rho}{\partial x}  = 0$, where $c\in \mathbb{R}$, determine $\rho(x,t)$ if $\rho(x,0) = \sin(x)$ by finding the characteristics, and giving a value for $\rho(x,t)$ along each characteristic. 

For more on finite difference simulation of PDEs with Python, see 

https://hplgit.github.io/fdm-book/doc/pub/book/sphinx/._book011.html